In [27]:
import os
import sys
!{sys.executable} -m pip install mlxtend
!{sys.executable} -m pip install numpy

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten, Conv2D, MaxPooling2D # new!
from mlxtend.data import loadlocal_mnist
from random import random
from math import floor

In [2]:
X_train, y_train = loadlocal_mnist(
        images_path=os.path.abspath('') + '/emnist-letters-train-images-idx3-ubyte', 
        labels_path=os.path.abspath('') + '/emnist-letters-train-labels-idx1-ubyte')

In [3]:
X_test, y_test = loadlocal_mnist(
        images_path=os.path.abspath('') + '/emnist-letters-test-images-idx3-ubyte', 
        labels_path=os.path.abspath('') + '/emnist-letters-test-labels-idx1-ubyte')

In [4]:
X_train = X_train.reshape(124800, 28, 28, 1).astype('float32')
X_test = X_test.reshape(20800, 28, 28, 1).astype('float32')


In [5]:
X_train /= 255
X_test /= 255

In [6]:
random_idx = floor(random() * len(X_train))
X_0 = X_train[random_idx]
for i in range(len(X_0)):
  for j in range(len(X_0)):
    if X_0[j][i][0] == 0:
      print(" ", end="")
    elif X_0[j][i][0] < 0.1:
      print("x", end="")
    else:
      print("X", end="")
  print("")
print(y_train[random_idx], chr(ord('a') + y_train[random_idx] - 1))

                            
                            
                            
                            
                            
             xxx            
         xxxxxXXxx          
       xXXXXXXXXXXx         
      xxXXXXXXXXXXx         
     xxXXXXXXXXXXXXx        
     xXXXXXXXXXXXXXx        
   xxXXXXXXXXXXXXXXx        
   xXXXXXXXXXXXXXXXx        
  xXXXXXXXXXXXXXXXx         
  xXXXXXXXXXXXXXXX  xxx     
  xXXXXXXXXXxxxxx xxXXXXx   
  xXXXXXxxxxx     xXXXXXXx  
  xXXXXXxxxxxxxxXXXXXXXXXx  
  xXXXXXXXXXXXXXXXXXXXXXXx  
  xXXXXXXXXXXXXXXXXXXXXXx   
   XXXXXXXXXXXXXXXXXXXXxx   
    xxXXXXXXXXXXXXxxxx      
     xxxxxxXXXXXxxx         
                            
                            
                            
                            
                            
5 e


In [9]:
# Subtract 1 because dataset is not 0-indexed
y_train -= 1
y_test -= 1

n_classes = 26
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)


In [13]:
print(y_train[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  0.]


array([[  7.90849555e-08,   9.90288740e-10,   4.76160825e-14, ...,
          6.23440836e-04,   5.46017587e-02,   7.42357402e-13],
       [  6.23273736e-05,   1.53331389e-03,   3.85592375e-06, ...,
          1.89356723e-13,   4.15856283e-09,   4.24175417e-10],
       [  1.04273399e-08,   4.21245764e-12,   4.52261601e-13, ...,
          1.92635164e-14,   2.01463360e-10,   2.45330687e-12],
       ..., 
       [  1.55104089e-11,   1.47427491e-18,   1.75851548e-26, ...,
          7.98157384e-13,   3.16413088e-14,   2.42422813e-22],
       [  2.62128901e-06,   8.32449416e-07,   6.05411988e-05, ...,
          8.31233453e-16,   1.05873132e-09,   3.93642202e-11],
       [  3.01207913e-11,   7.49735182e-06,   3.09967746e-11, ...,
          9.40657299e-16,   3.45178219e-09,   2.63387524e-11]], dtype=float32)

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (28, 28, 1)

In [15]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test))

Train on 124800 samples, validate on 20800 samples
Epoch 1/10
124800/124800 [==============================] - 174s 1ms/step - loss: 0.7774 - acc: 0.7620 - val_loss: 0.3009 - val_acc: 0.9033
Epoch 2/10
124800/124800 [==============================] - 175s 1ms/step - loss: 0.4230 - acc: 0.8657 - val_loss: 0.2456 - val_acc: 0.9215
Epoch 3/10
124800/124800 [==============================] - 174s 1ms/step - loss: 0.3461 - acc: 0.8876 - val_loss: 0.2261 - val_acc: 0.9262
Epoch 4/10
124800/124800 [==============================] - 172s 1ms/step - loss: 0.3072 - acc: 0.8999 - val_loss: 0.2076 - val_acc: 0.9323
Epoch 5/10
124800/124800 [==============================] - 177s 1ms/step - loss: 0.2789 - acc: 0.9088 - val_loss: 0.2001 - val_acc: 0.9346
Epoch 6/10
124800/124800 [==============================] - 173s 1ms/step - loss: 0.2552 - acc: 0.9145 - val_loss: 0.1997 - val_acc: 0.9355
Epoch 7/10
124800/124800 [==============================] - 174s 1ms/step - loss: 0.2413 - acc: 0.9195 - val_

In [ ]:
model.save('letters.h5')

In [40]:
max([ (x, i) for (i, x) in enumerate(model.predict(np.expand_dims(X_test[3], axis=0))[0])])

(0.63313138, 0)

In [52]:
random_idx = floor(random() * len(X_train))

best_so_far = None
for (i, x) in enumerate(model.predict(np.expand_dims(X_train[random_idx], axis=0))[0]):
    if not best_so_far:
        best_so_far = (i, x)
    if x > best_so_far[1]:
        best_so_far = (i, x)
    print(i, x)
print(best_so_far, list(y_train[random_idx]).index(1))    

0 6.126e-06
1 2.22252e-07
2 6.87693e-08
3 6.53856e-08
4 5.59952e-07
5 4.63372e-09
6 4.88978e-07
7 9.96623e-12
8 4.26259e-06
9 1.61046e-08
10 3.50283e-09
11 2.63633e-07
12 9.87189e-17
13 4.61695e-10
14 3.58262e-12
15 2.87285e-08
16 1.59056e-08
17 1.72544e-06
18 4.7496e-10
19 5.78652e-07
20 1.06813e-10
21 1.25534e-11
22 8.69483e-14
23 3.11338e-08
24 8.03266e-09
25 0.999986
(25, 0.99998558) 25


In [29]:
y_train[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.], dtype=float32)

In [ ]:
0

In [ ]:
4

In [ ]:
print(X_train[0])